# Этап 0. Импорт библиотек и описание процесса решения задачи

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

tqdm.pandas()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Rumatica/Август/август2/Подготовленные для заполнения типов и категорий данные.csv', sep=';')
data.head()

,Unnamed: 0,product_type_ru,product_type_en,product_type_de,product_category,photo,description_ru,description_de,description_und
0,0,Защитный выключатель блокировки,Guard locking safety switch,NaN,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,-,-,", safety switch: blocking; slc; contacts: nc+n..."
1,1,Защитный выключатель блокировки,Guard locking safety switch,NaN,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,-,-,model
2,2,Защитный выключатель блокировки,Guard locking safety switch,NaN,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,-,-,1. safety interlock switch (6018200008) - 1 pc.
3,3,NaN,Switches with guard locking,NaN,Переключатели,https://www.bernstein.eu/fileadmin/_migrated/p...,переключатель \nпоследующая модель для 6018119...,", sicherheitschalter [ nachfolger von 6018119...","germany limit sw slk-fvtu24uc55arx ho,sicheru..."
4,4,NaN,Switches with guard locking,NaN,Переключатели,https://www.bernstein.eu/fileadmin/_migrated/p...,выключатель a,-,a


# Этап 1. Дозаполнение признаков и поиск категорий в ключевых словах

Удаляем ненужный признак, дублирующий индексы


In [ ]:
data = data.drop('Unnamed: 0', axis=1)
data.head(1)

,product_type_ru,product_type_en,product_type_de,product_category,photo,description_ru,description_de,description_und
0,Защитный выключатель блокировки,Guard locking safety switch,NaN,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,-,-,", safety switch: blocking; slc; contacts: nc+n..."


Заполним пропущенные данные в столбцах с типом продукта

In [ ]:
type_cols = ['product_type_ru',	'product_type_en',	'product_type_de']
for col in type_cols:
  data[col] = data[col].fillna('-')
data.head()

,product_type_ru,product_type_en,product_type_de,product_category,photo,description_ru,description_de,description_und
0,Защитный выключатель блокировки,Guard locking safety switch,-,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,-,-,", safety switch: blocking; slc; contacts: nc+n..."
1,Защитный выключатель блокировки,Guard locking safety switch,-,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,-,-,model
2,Защитный выключатель блокировки,Guard locking safety switch,-,Переключатели,https://www.bernstein.eu/fileadmin/img/produkt...,-,-,1. safety interlock switch (6018200008) - 1 pc.
3,-,Switches with guard locking,-,Переключатели,https://www.bernstein.eu/fileadmin/_migrated/p...,переключатель \nпоследующая модель для 6018119...,", sicherheitschalter [ nachfolger von 6018119...","germany limit sw slk-fvtu24uc55arx ho,sicheru..."
4,-,Switches with guard locking,-,Переключатели,https://www.bernstein.eu/fileadmin/_migrated/p...,выключатель a,-,a


Получим названия типов и названия описаний для русского языка

In [ ]:
product_type_ru = set(data['product_type_ru'].apply(lambda x: str(x).lower()).to_list())
description_ru = list()

for y in tqdm(data['description_ru'].to_list()):
  description_ru.extend([x.lstrip().rstrip() for x in re.split(": |; |, |\. |:|;|,|\.", str(y)) if len(x.lstrip().rstrip()) > 1])

100%|██████████| 356960/356960 [00:04<00:00, 88219.06it/s] 


Получим список типов продуктов на руском, которые встречаются в ключевых словах

In [ ]:
russian_types = []
for type_p in tqdm(product_type_ru):
  if type_p in description_ru:
    russian_types.append(type_p)
  else:
    continue

print(russian_types)

100%|██████████| 1381/1381 [00:46<00:00, 29.69it/s]

['ультразвуковой датчик расстояний', 'датчик давления расплава', 'разъем для соленоида', 'комплект для ремонта', 'маслораспылитель', 'бесконтактный индуктивный датчик', 'емкостной датчик предельного уровня', 'датчики', 'плк', 'пневмопривод', 'соединительное изделие', 'оптоволоконный датчик', 'толкатель кнопок', 'плунжер', 'защита от перенапряжения', 'регулятор температуры', 'датчик индуктивный', 'реле перегрузки', 'шарнирное крепление', 'элемент фильтра', 'фильтр', 'волоконно-оптический датчик', 'реле давления воздуха', 'роликоподшипник игольчатый', 'насос шестеренный', 'адаптер боковой нагрузки', 'регулятор расхода', 'поршень', 'монтажный комплект', 'держатель', 'измерительный преобразователь', 'позиционер электропневматический', 'направляющая', 'термопреобразователь', 'крепежный кронштейн', 'датчик угла поворота', 'главный выключатель', 'поворотный переключатель', 'катушка', 'картридж', 'вибромагнит', 'электромагниты', 'модуль подключения', 'линейный датчик', 'седло клапана', 'датчик

In [ ]:
data2 = data.copy()

Проведем обратную вставку типа по ключевому слову


In [ ]:
for i in tqdm(range(data.shape[0])):
  for x in russian_types:
    if data['description_ru'].iloc[i] == '-':
      break
    elif x in str(data['description_ru'].iloc[i]) and str(data['product_type_ru'].iloc[i]) == '-':
      data['product_type_ru'].iloc[i] = x
    else:
      continue

100%|██████████| 356960/356960 [1:20:10<00:00, 74.20it/s] 


In [ ]:
data.to_csv('Заполненные категории на русском.csv', index=False)

In [ ]:
data['product_type_ru'] = data['product_type_ru'].apply(lambda x: x.capitalize())

Посмотрим различие между числом незаполненных значений до и после обработки

In [ ]:
display(data2['product_type_ru'].value_counts())
display(data['product_type_ru'].value_counts())

,count
product_type_ru,
-,337120
Монтажный кронштейн,1060
Cоединитель,854
Индуктивный датчик,633
Энкодер,611
...,...
Защелкивающийся замок,1
Рычажок,1
Изоляционный корпус,1


,count
product_type_ru,
-,159789
Клапан,12161
Фильтр,7938
Ключ,7763
Насос,6972
...,...
Реле сцепления,1
Блок подготовки,1
2-ходовой шаровый кран,1


In [ ]:
data.to_csv('Заполненные категории на русском.csv', index=False)

In [28]:
cat_ru_data = set(data['product_type_ru'].to_list())
cat_ru_data2 = set(data2['product_type_ru'].to_list())
print(len(cat_ru_data.intersection(cat_ru_data2)))

1322
